In [ ]:
!nvidia-smi

Sat Nov 21 03:39:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/baomoi.window2.vn.model.bin.gz /content/baomoi.window2.vn.model.bin.gz

In [ ]:
import gensim

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("/content/baomoi.window2.vn.model.bin.gz", binary=True)

In [ ]:
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torchtext.data import Field, TabularDataset, BucketIterator
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("/content/DEVC_DATA.csv")
data = data.dropna()
labels = [int(x) for x in data.label.values]
positive_label = 0
negative_label = 0
neural_label = 0
for label in labels:
  if label == 0:
    positive_label += 1
  elif label == 2:
    negative_label += 1
  else:
    neural_label += 1  
print(set(labels))
print(positive_label)
print(negative_label)
print(neural_label)

{0, 1, 2}
45727
13367
19764


In [ ]:
train, test = train_test_split(data, test_size = 0.3, random_state = 42)
train["label"] = train["label"].apply(lambda x: int(x))
test["label"] = test["label"].apply(lambda x: int(x))
train.to_csv("/content/train.csv", index=False)
test.to_csv("/content/test.csv", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
train

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.int)
text_field = Field(sequential=True, include_lengths=True, use_vocab=True, batch_first=True)
fields = [("text", text_field),("label", label_field)]
train, test = TabularDataset.splits(path='/content/', train='train.csv', test='test.csv',
                                           format='CSV', fields=fields, skip_header=True)
train_iter = BucketIterator(train, batch_size=32, sort_key=lambda x: len(x.text),
                            device=device, sort=True, sort_within_batch=True)
test_iter = BucketIterator(test, batch_size=32, sort_key=lambda x: len(x.text),
                            device=device, sort=True, sort_within_batch=True)
text_field.build_vocab(train, min_freq=2)
vocab_size = len(text_field.vocab.stoi)

In [ ]:
import pickle
def save_vocab(vocab, path):
    vocab_dict = dict()
    vocab_file = open(path, "wb")
    for token, index in vocab.stoi.items():
        vocab_dict[token] = index
    pickle.dump(vocab_dict, vocab_file)
def read_vocab(path):
    vocab_file = open(path, "rb")
    vocab = pickle.load(vocab_file)
    return vocab
def predict_sentiment(model, sentence, vocab, unk_idx):

    tokens = tokenize(sentence) #convert string to tokens, needs to be same tokenization as training
    indexes = [vocab.get(t, unk_idx) for t in tokens] #converts to index or unk if not in vocab
    tensor = torch.LongTensor(indexes).unsqueeze(1) #convert to tensor and add batch dimension
    output = model(tensor) #get output from model
    prediction = torch.sigmoid(output) #squeeze between 0-1 range
    return prediction
save_vocab(text_field.vocab, "/content/vocab.pkl")

In [ ]:
vocab = read_vocab("/content/vocab.pkl")
print(len(vocab))

18959


In [ ]:
pretrained_vectors = []
for token, id in text_field.vocab.stoi.items():
  if token in model.vocab.keys():
    pretrained_vectors.append(torch.FloatTensor(torch.from_numpy(model[token])))
  else:
    pretrained_vectors.append(torch.zeros(300))
text_field.vocab.set_vectors(text_field.vocab.stoi, pretrained_vectors, 300)
pretrained_embedding = torch.FloatTensor(text_field.vocab.vectors)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  after removing the cwd from sys.path.


In [ ]:
class VSA_BiLSTM(nn.Module):
  def __init__(self, pretrained_emb, emb_size=300, hidden_size=512):
    super(VSA_BiLSTM, self).__init__()
    self.embedding = nn.Embedding.from_pretrained(pretrained_embedding)
    # self.embedding = nn.Embedding(vocab_size, emb_size)
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size=emb_size,
                        hidden_size=self.hidden_size,
                        num_layers=1,
                        batch_first=True,
                        bidirectional=True)
    self.dropout = nn.Dropout(p=0.1)

    self.fc = nn.Linear(2*self.hidden_size,3)
  def forward(self, text):
    #Text = [Batch_size, seq_len]
    text_emb = self.embedding(text)
    #Text_emb = [Batch_size, seq_len, emb_dim]
    output, (hidden, cell) = self.lstm(text_emb)
    #output = [Batch_Size, seq_len, hidden_size*directions]
    output_forward = torch.squeeze(output[:, -1, :self.hidden_size], 1)
    output_backward = torch.squeeze(output[:, -1, self.hidden_size:], 1)
    out_bidirectional = torch.cat((output_forward, output_backward), 1)
    out_dropped = self.dropout(out_bidirectional)
    return self.fc(out_dropped)

In [ ]:
class VSA_BiLSTM_sa(nn.Module):
  def __init__(self, pretrained_emb, emb_size=300, hidden_size=512, d_a=350, r=30):
    super(VSA_BiLSTM_sa, self).__init__()
    # self.embedding = nn.Embedding.from_pretrained(pretrained_embedding)
    self.embedding = nn.Embedding(vocab_size, emb_size)
    self.hidden_size = hidden_size
    self.r = r
    self.d_a = d_a
    self.lstm = nn.LSTM(input_size=emb_size,
                        hidden_size=self.hidden_size,
                        num_layers=1,
                        batch_first=True,
                        bidirectional=True)
    self.linear_first = torch.nn.Linear(self.hidden_size*2,self.d_a)
    self.linear_first.bias.data.fill_(0)
    self.linear_second = torch.nn.Linear(self.d_a,self.r)
    self.linear_second.bias.data.fill_(0)
    self.linear_final = torch.nn.Linear(self.hidden_size*2,3)

  def forward(self, text):
    #Text = [Batch_size, seq_len]
    text_emb = self.embedding(text)
    #Text_emb = [Batch_size, seq_len, emb_dim]
    output, (hidden, cell) = self.lstm(text_emb)
    #output = [Batch_Size, seq_len, hidden_size*directions]
    x = F.tanh(self.linear_first(output))
    #x = [Batch_size, seq_len, d_a]
    x = self.linear_second(x)
    #x = [Batch_size, seq_len, r]
    x = self.softmax(x, axis = 1)
    #x = [Batch_size, seq_len, r]
    attention = x.transpose(1,2)   
    #x = [Batch_size, r, seq_len]    
    sentence_embeddings = attention@output
    #x = [Batch_size, r, hiddensize*2]    
    avg_sentence_embeddings = torch.sum(sentence_embeddings,1)/self.r
    return self.linear_final(avg_sentence_embeddings)
    
    

  def softmax(self,input, axis=1):

      input_size = input.size()
      trans_input = input.transpose(axis, len(input_size)-1)
      trans_size = trans_input.size()
      input_2d = trans_input.contiguous().view(-1, trans_size[-1])
      soft_max_2d = F.softmax(input_2d)
      soft_max_nd = soft_max_2d.view(*trans_size)
      return soft_max_nd.transpose(axis, len(input_size)-1)
                                 

In [ ]:
Dataloader = {"train": train_iter, "val": test_iter}
vsa_bilstm_sa = VSA_BiLSTM_sa(pretrained_embedding).to(device)
optimizer = optim.Adam(vsa_bilstm_sa.parameters(), lr=0.0005)

In [ ]:
import time
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
total_step = len(Dataloader["train"])
def train_model(model, optimizer, device, num_epochs=300):
    since = time.time()
    print("Start training process")
    for epoch in range(1,num_epochs + 1):
        loss_set = []
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        Pred_lst = []
        Label_lst = []
        for phase in ["train", "val"]:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
            for i, batch in enumerate(Dataloader[phase]):
                texts, texts_len = batch.text
                len_sen = texts.size()[1]
                # if len_sen <= 5:
                #   pad = torch.zeros((32, 5-len_sen))
                #   texts = torch.cat((texts.cuda(), pad.cuda()), 1)
                labels = batch.label
                texts = texts.long().to(device)
                labels = labels.long().to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(texts)
                    loss = F.cross_entropy(outputs, labels)
                    if phase == "val":
                      prediction = torch.argmax(outputs, dim = 1)
                      for ele in prediction.detach().cpu().numpy():
                        Pred_lst.append(ele)
                      for ele in labels.detach().cpu().numpy():
                        Label_lst.append(ele)
                    if phase == 'train':
                        loss_set.append(loss.item())
                        loss.backward()
                        optimizer.step()
                if (i % 50) == 0:
                  print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, num_epochs, i+1, total_step, loss.item()))
            if phase == "train":
              print(" Training process of epoch {} has been done  ------- Loss_train: {}".format(epoch, loss_set[-1]))
            elif phase == "val":
              Pred_lst = [int(ele) for ele in Pred_lst]
              Label_lst = [int(ele) for ele in Label_lst]
              print(" Validation process of epoch {} has been done -- F1_score_val: {}% -- Acc_score: {}%".format(epoch, 100*f1_score(Pred_lst, Label_lst, labels=[0,1,2], average="weighted"), 100*accuracy_score(Label_lst, Pred_lst)))         
        if False:
          path = "/content/vsa_lstm_sa/" + "BiLSTM_Classification_" + str(epoch) + ".pth"
          torch.save(model.state_dict(), path)   
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [ ]:
train_model(vsa_bilstm_sa, optimizer, device)

Start training process
Epoch 1/300
----------
Epoch [1/300], Step [1/1725], Loss: 1.1042


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/300], Step [51/1725], Loss: 0.9764
Epoch [1/300], Step [101/1725], Loss: 0.6900
Epoch [1/300], Step [151/1725], Loss: 0.4212
Epoch [1/300], Step [201/1725], Loss: 0.6184
Epoch [1/300], Step [251/1725], Loss: 0.4079
Epoch [1/300], Step [301/1725], Loss: 0.4783
Epoch [1/300], Step [351/1725], Loss: 0.7679
Epoch [1/300], Step [401/1725], Loss: 0.6810
Epoch [1/300], Step [451/1725], Loss: 0.5477
Epoch [1/300], Step [501/1725], Loss: 0.6384
Epoch [1/300], Step [551/1725], Loss: 0.6625
Epoch [1/300], Step [601/1725], Loss: 0.6591
Epoch [1/300], Step [651/1725], Loss: 0.4910
Epoch [1/300], Step [701/1725], Loss: 0.6910
Epoch [1/300], Step [751/1725], Loss: 0.5837
Epoch [1/300], Step [801/1725], Loss: 0.5230
Epoch [1/300], Step [851/1725], Loss: 0.7867
Epoch [1/300], Step [901/1725], Loss: 0.8356
Epoch [1/300], Step [951/1725], Loss: 0.6639
Epoch [1/300], Step [1001/1725], Loss: 0.6987
Epoch [1/300], Step [1051/1725], Loss: 0.6169
Epoch [1/300], Step [1101/1725], Loss: 0.6888
Epoch [1

KeyboardInterrupt: ignored

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]

        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(text_field.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 3
DROPOUT = 0.5
PAD_IDX = text_field.vocab.stoi[text_field.pad_token]
CNN_1 = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
pretrained_embeddings = text_field.vocab.vectors
CNN_1.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
UNK_IDX = text_field.vocab.stoi[text_field.unk_token]

CNN_1.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
CNN_1.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()
CNN_1 = CNN_1.to(device)
criterion = criterion.to(device)

In [ ]:
CNN_1.cuda()

In [ ]:
train_model(CNN_1, optimizer, "cuda")

In [ ]:
vsa_bilstm.cpu()
def read_vocab(path):
    vocab_file = open(path, "rb")
    vocab = pickle.load(vocab_file)
    return vocab
vocab = read_vocab("/content/vocab.pkl")
def predict(model, text):
    indexed = [int(vocab[token]) if token in vocab.keys() else 0 for token in text]
    tensor = torch.LongTensor(indexed).unsqueeze(0)
    label = int(torch.argmax(model(tensor),dim = 1).cpu().numpy().item())
    return label

In [ ]:
import time

In [ ]:
start = time.time()
predict(vsa_bilstm, "Alo sản_phẩm này sao tệ thế hả")
print("the inference time is: ", time.time()-start)